#### Import the necessary package

In [1]:
import make_env as world
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import random

Using TensorFlow backend.


#### load the environment

In [2]:
env = world.make_env('simple')

In [3]:
env.discrete_action_input = True
for i in range(100):
    val =  env.render(mode='Test')
    val = env.step(action_n=[np.random.choice([0,1,2,3,4])])

#### Create the model

In [4]:
agent_move = Sequential()

In [5]:
agent_move.add(Dense(64,activation='relu', input_dim = 5))
agent_move.add(Dense(50,activation='relu'))
agent_move.add(Dense(32,activation='relu'))
agent_move.add(Dense(24,activation='relu'))
agent_move.add(Dense(16,activation='relu'))
agent_move.add(Dense(10,activation='relu'))
agent_move.add(Dense(8, activation='relu'))
agent_move.add(Dense(4,activation='relu'))
agent_move.add(Dense(1,activation='linear'))

In [6]:
agent_move.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                384       
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_4 (Dense)              (None, 24)                792       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                400       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                170       
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 88        
__________

In [7]:
agent_move.compile(loss='mean_squared_error', optimizer='Adam')

#### Introduce target network

In [7]:
agent_target = Sequential()

In [8]:
agent_target.add(Dense(64,activation='relu', input_dim = 5))
agent_target.add(Dense(50,activation='relu'))
agent_target.add(Dense(32,activation='relu'))
agent_target.add(Dense(24,activation='relu'))
agent_target.add(Dense(16,activation='relu'))
agent_target.add(Dense(10,activation='relu'))
agent_target.add(Dense(8, activation='relu'))
agent_target.add(Dense(4,activation='relu'))
agent_target.add(Dense(1,activation='linear'))

In [9]:
agent_target.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 64)                384       
_________________________________________________________________
dense_11 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_12 (Dense)             (None, 32)                1632      
_________________________________________________________________
dense_13 (Dense)             (None, 24)                792       
_________________________________________________________________
dense_14 (Dense)             (None, 16)                400       
_________________________________________________________________
dense_15 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 88        
__________

In [11]:
agent_move.save_weights('./random_weight')

In [155]:
episode = 1000
max_episode_length = 400
discount_factor = 0.9

In [168]:
for i in range(276,episode):
    
    #replace the experience replay 
    if ((i+1) % 25) == 0:
        episode_experience = episode_experience[-2000:]
        
        #replace old network with new network
        agent_move.save_weights('./weight')
        
        agent_target.load_weights('./weight')
        
        agent_move.load_weights('./random_weight')
    
    if i == 0:
        episode_experience = []
    
    
    epsilon = 1 / (np.sqrt(i+10))
    
    S = env.reset()
    S = S[0]
    
    A = np.random.choice([0,1,2,3,4])

    for j in range(max_episode_length):
        
        observe = env.step(action_n=[A])
        
        # Reward clip so that it values from 1 to -1
        R = observe[1][0] 
        
        if R < -1.5:
            R = -1
        else :
            R = (1.5 + R) / (1.5)
        
        S_new = observe[0][0]
        
        Test = np.array([[S_new[0],S_new[1],S_new[2],S_new[3], 0],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 1],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 2],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 3],
                         [S_new[0],S_new[1],S_new[2],S_new[3], 4]])
        
        #value of all possible action from target network
        all_action_value = agent_move.predict(Test)
        
        #chose action which is maximum value 
        A_new = np.argmax(all_action_value)
        
        
        #epsilon - greedy 
        rand = np.random.uniform([0])
        
        if rand[0] > epsilon:
            A_new = A_new
        else :
            A_new = np.random.choice([0,1,2,3,4])
        
        
        Target = agent_target.predict(Test)[A_new][0]
        
        
        episode_experience.append([S[0],S[1],S[2],S[3],A,R + discount_factor * Target])
            
        S = S_new
        A = A_new
        
        if (j % (200)) == 0 :
            env.reset()
            
            S = env.reset()
            S = S[0]
    
            A = np.random.choice([0,1,2,3,4])
    
        if (i == 0 and j > 65) or (i > 0) :
            minibatch = random.sample(episode_experience,64)
    
            Test = np.array(minibatch)
        
            agent_move.fit(Test[:,0:5], Test[:,5], epochs=3)
    

Epoch 1/3
64/64 [==============================] - 0s 699us/step - loss: 0.1422
Epoch 2/3
64/64 [==============================] - 0s 888us/step - loss: 0.1334
Epoch 3/3
64/64 [==============================] - 0s 662us/step - loss: 0.1166
Epoch 1/3
64/64 [==============================] - 0s 603us/step - loss: 0.0847
Epoch 2/3
64/64 [==============================] - 0s 682us/step - loss: 0.0806
Epoch 3/3
64/64 [==============================] - 0s 653us/step - loss: 0.0734
Epoch 1/3
64/64 [==============================] - 0s 605us/step - loss: 0.1571
Epoch 2/3
64/64 [==============================] - 0s 783us/step - loss: 0.0734
Epoch 3/3
64/64 [==============================] - 0s 733us/step - loss: 0.0716
Epoch 1/3
64/64 [==============================] - 0s 766us/step - loss: 0.2776
Epoch 2/3
64/64 [==============================] - 0s 932us/step - loss: 0.2678
Epoch 3/3
64/64 [==============================] - 0s 662us/step - loss: 0.2474
Epoch 1/3
64/64 [=======================

64/64 [==============================] - 0s 763us/step - loss: 0.0772
Epoch 2/3
64/64 [==============================] - 0s 816us/step - loss: 0.0618
Epoch 3/3
64/64 [==============================] - 0s 629us/step - loss: 0.0721
Epoch 1/3
64/64 [==============================] - 0s 655us/step - loss: 0.1110
Epoch 2/3
64/64 [==============================] - 0s 798us/step - loss: 0.1100
Epoch 3/3
64/64 [==============================] - 0s 771us/step - loss: 0.0990
Epoch 1/3
64/64 [==============================] - 0s 847us/step - loss: 0.0302
Epoch 2/3
64/64 [==============================] - 0s 739us/step - loss: 0.0302
Epoch 3/3
64/64 [==============================] - 0s 648us/step - loss: 0.0288
Epoch 1/3
64/64 [==============================] - 0s 715us/step - loss: 0.1002
Epoch 2/3
64/64 [==============================] - 0s 818us/step - loss: 0.0769
Epoch 3/3
64/64 [==============================] - 0s 631us/step - loss: 0.0595
Epoch 1/3
64/64 [==============================] -

64/64 [==============================] - 0s 915us/step - loss: 0.1938
Epoch 2/3
64/64 [==============================] - 0s 1ms/step - loss: 0.1320
Epoch 3/3
64/64 [==============================] - 0s 804us/step - loss: 0.1028
Epoch 1/3
64/64 [==============================] - 0s 904us/step - loss: 0.2424
Epoch 2/3
64/64 [==============================] - 0s 593us/step - loss: 0.1832
Epoch 3/3
64/64 [==============================] - 0s 860us/step - loss: 0.1543
Epoch 1/3
64/64 [==============================] - 0s 1ms/step - loss: 0.0819
Epoch 2/3
64/64 [==============================] - 0s 982us/step - loss: 0.0843
Epoch 3/3
64/64 [==============================] - 0s 763us/step - loss: 0.0751
Epoch 1/3
64/64 [==============================] - 0s 780us/step - loss: 0.0897
Epoch 2/3
64/64 [==============================] - 0s 765us/step - loss: 0.0851
Epoch 3/3
64/64 [==============================] - 0s 625us/step - loss: 0.0821
Epoch 1/3
64/64 [==============================] - 0s 

64/64 [==============================] - 0s 606us/step - loss: 0.0929
Epoch 2/3
64/64 [==============================] - 0s 604us/step - loss: 0.0835
Epoch 3/3
64/64 [==============================] - 0s 455us/step - loss: 0.0697
Epoch 1/3
64/64 [==============================] - 0s 512us/step - loss: 0.0399
Epoch 2/3
64/64 [==============================] - 0s 588us/step - loss: 0.0387
Epoch 3/3
64/64 [==============================] - 0s 530us/step - loss: 0.0381
Epoch 1/3
64/64 [==============================] - 0s 593us/step - loss: 0.0676
Epoch 2/3
64/64 [==============================] - 0s 573us/step - loss: 0.0536
Epoch 3/3
64/64 [==============================] - 0s 535us/step - loss: 0.0580
Epoch 1/3
64/64 [==============================] - 0s 618us/step - loss: 0.3896
Epoch 2/3
64/64 [==============================] - 0s 1ms/step - loss: 0.2767
Epoch 3/3
64/64 [==============================] - 0s 903us/step - loss: 0.1815
Epoch 1/3
64/64 [==============================] - 0

64/64 [==============================] - 0s 780us/step - loss: 0.0648
Epoch 2/3
64/64 [==============================] - 0s 655us/step - loss: 0.0565
Epoch 3/3
64/64 [==============================] - 0s 706us/step - loss: 0.0447
Epoch 1/3
64/64 [==============================] - 0s 850us/step - loss: 0.1718
Epoch 2/3
64/64 [==============================] - 0s 735us/step - loss: 0.1458
Epoch 3/3
64/64 [==============================] - 0s 981us/step - loss: 0.1328
Epoch 1/3
64/64 [==============================] - 0s 1ms/step - loss: 0.0671
Epoch 2/3
64/64 [==============================] - 0s 1ms/step - loss: 0.0534
Epoch 3/3
64/64 [==============================] - 0s 1ms/step - loss: 0.0462
Epoch 1/3
64/64 [==============================] - 0s 998us/step - loss: 0.0916
Epoch 2/3
64/64 [==============================] - 0s 1ms/step - loss: 0.0886
Epoch 3/3
64/64 [==============================] - 0s 1ms/step - loss: 0.0805
Epoch 1/3
64/64 [==============================] - 0s 1ms/st

...

64/64 [==============================] - 0s 632us/step - loss: 0.0264
Epoch 3/3
64/64 [==============================] - 0s 486us/step - loss: 0.0220
Epoch 1/3
64/64 [==============================] - 0s 550us/step - loss: 0.0502
Epoch 2/3
64/64 [==============================] - 0s 638us/step - loss: 0.0377
Epoch 3/3
64/64 [==============================] - 0s 578us/step - loss: 0.0307
Epoch 1/3
64/64 [==============================] - 0s 649us/step - loss: 0.0239
Epoch 2/3
64/64 [==============================] - 0s 599us/step - loss: 0.0219
Epoch 3/3
64/64 [==============================] - 0s 742us/step - loss: 0.0193
Epoch 1/3
64/64 [==============================] - 0s 721us/step - loss: 0.0754
Epoch 2/3
64/64 [==============================] - 0s 809us/step - loss: 0.0332
Epoch 3/3
64/64 [==============================] - 0s 818us/step - loss: 0.0289
Epoch 1/3
64/64 [==============================] - 0s 671us/step - loss: 0.0343
Epoch 2/3
64/64 [==============================] -

64/64 [==============================] - 0s 792us/step - loss: 19.6230
Epoch 1/3
64/64 [==============================] - 0s 343us/step - loss: 19.6037
Epoch 2/3
64/64 [==============================] - 0s 769us/step - loss: 19.5712
Epoch 3/3
64/64 [==============================] - 0s 587us/step - loss: 19.5440
Epoch 1/3
64/64 [==============================] - 0s 482us/step - loss: 20.4351
Epoch 2/3
64/64 [==============================] - 0s 633us/step - loss: 20.4035
Epoch 3/3
64/64 [==============================] - 0s 596us/step - loss: 20.3726
Epoch 1/3
64/64 [==============================] - 0s 466us/step - loss: 20.4057
Epoch 2/3
64/64 [==============================] - 0s 715us/step - loss: 20.3739
Epoch 3/3
64/64 [==============================] - 0s 650us/step - loss: 20.3420
Epoch 1/3
64/64 [==============================] - 0s 547us/step - loss: 19.0322
Epoch 2/3
64/64 [==============================] - 0s 878us/step - loss: 18.9971
Epoch 3/3
64/64 [=====================

64/64 [==============================] - 0s 621us/step - loss: 14.7398
Epoch 2/3
64/64 [==============================] - 0s 607us/step - loss: 14.7178
Epoch 3/3
64/64 [==============================] - 0s 635us/step - loss: 14.7119
Epoch 1/3
64/64 [==============================] - 0s 550us/step - loss: 15.6426
Epoch 2/3
64/64 [==============================] - 0s 714us/step - loss: 15.6220
Epoch 3/3
64/64 [==============================] - 0s 534us/step - loss: 15.5978
Epoch 1/3
64/64 [==============================] - 0s 559us/step - loss: 14.4597
Epoch 2/3
64/64 [==============================] - 0s 626us/step - loss: 14.4386
Epoch 3/3
64/64 [==============================] - 0s 701us/step - loss: 14.4153
Epoch 1/3
64/64 [==============================] - 0s 523us/step - loss: 14.5159
Epoch 2/3
64/64 [==============================] - 0s 826us/step - loss: 14.3767
Epoch 3/3
64/64 [==============================] - 0s 716us/step - loss: 14.2969
Epoch 1/3
64/64 [=====================

64/64 [==============================] - 0s 669us/step - loss: 11.8875
Epoch 3/3
64/64 [==============================] - 0s 595us/step - loss: 11.8730
Epoch 1/3
64/64 [==============================] - 0s 506us/step - loss: 12.0286
Epoch 2/3
64/64 [==============================] - 0s 512us/step - loss: 12.0165
Epoch 3/3
64/64 [==============================] - 0s 574us/step - loss: 12.0034
Epoch 1/3
64/64 [==============================] - 0s 1ms/step - loss: 12.3694
Epoch 2/3
64/64 [==============================] - 0s 956us/step - loss: 12.3582
Epoch 3/3
64/64 [==============================] - 0s 1ms/step - loss: 12.3467
Epoch 1/3
64/64 [==============================] - 0s 708us/step - loss: 12.0383
Epoch 2/3
64/64 [==============================] - 0s 882us/step - loss: 12.0258
Epoch 3/3
64/64 [==============================] - 0s 473us/step - loss: 12.0131
Epoch 1/3
64/64 [==============================] - 0s 544us/step - loss: 12.3477
Epoch 2/3
64/64 [=========================

KeyboardInterrupt: 

In [157]:
i

276

In [14]:

S = env.reset()
S = S[0]
    
A = np.random.choice([0,1,2,3,4])

epsilon = 0.0
agent_target.load_weights('./weight')
for _ in range(100):
    time.sleep(1)
    observe = env.step(action_n=[A])
        
    chek =  env.render(mode='Test')
        
    # Reward clip so that it values from 1 to -1
    R = observe[1][0] 
        
    if R < -1.5:
        R = -1
    else :
        R = (1.5 + R) / (1.5)
        
    S_new = observe[0][0]
        
    Test = np.array([[S_new[0],S_new[1],S_new[2],S_new[3], 0],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 1],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 2],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 3],
                    [S_new[0],S_new[1],S_new[2],S_new[3], 4]])
    
   
    #value of all possible action from target network
    all_action_value = agent_target.predict(Test)
        
    #chose action which is maximum value 
    A_new = np.argmax(all_action_value)
    
    rand = np.random.uniform([0])
    if rand[0] > epsilon:
            A_new = A_new
    else :
            A_new = np.random.choice([0,1,2,3,4])
    
    S = S_new
    A = A_new
        

KeyboardInterrupt: 

In [175]:
env2 = world.make_env('simple_adversary')

In [176]:
env2.render("test")

[None]

In [179]:
agent2 = env2.agents

In [200]:
env2.render('Test')
env.step(action_n=[3])

([array([-0.18677754, -1.79736273,  1.65011844,  1.35159726])],
 [-4.549706031478298],
 [False],
 {'n': [{}]})

In [44]:
for i in range(100):
    if (i%10) == 0:
        print(i,'1')
    
    if ((i+1) % 10) == 0:
        print(i, '2')

0 1
9 2
10 1
19 2
20 1
29 2
30 1
39 2
40 1
49 2
50 1
59 2
60 1
69 2
70 1
79 2
80 1
89 2
90 1
99 2


In [101]:
env.reset()

[array([ 0.        ,  0.        , -1.46814295,  0.59351347])]

In [10]:
import time

In [40]:
episode = [[1,2],[2,4],[3,1],[4,2]]

In [163]:
time.sleep(3)

In [41]:
random.sample(episode,3)

[[1, 2], [4, 2], [2, 4]]

In [167]:
i

276